See 1. A Numerical Example in Valuing American Options by Simulation: A Simple Least-Squares Approach Francis A. Longstaff UCLA Eduardo S. Schwartz UCLA.

In [15]:
import numpy as np
from sklearn.linear_model import LinearRegression

paths = np.array([[1.00, 1.09, 1.08, 1.34],
                 [1.00, 1.16, 1.26, 1.54],
                 [1.00, 1.22, 1.07, 1.03],
                 [1.00, 0.93, 0.97, 0.92],
                 [1.00, 1.11, 1.56, 1.52],
                 [1.00, 0.76, 0.77, 0.90],
                 [1.00, 0.92, 0.84, 1.01],
                 [1.00, 0.88, 1.22, 1.34]])

def least_square_Monte_Carlo_test(paths,r,dt,T,payoff,K):
    value = payoff(paths[:,-1])
    print(value)
    for col in paths.T[:-1][::-1]:# stepping backwards in time
       value *= np.exp(-r*dt) # discounting 
       ITMmask = col < K # selects paths ITM
       ITMpaths = col[ITMmask]
       ITMvalue = value[ITMmask]
       model = LinearRegression()
       X = np.column_stack((ITMpaths,ITMpaths**2))
       model.fit(X,ITMvalue) # LS of the continuation value
       interpolated = model.predict(X)
       exercised = payoff_put(ITMpaths,K)
    #  value[mask] = np.maximum(interpolated,exercise) see note in paper
       value[ITMmask] = np.where(interpolated>exercised, value[ITMmask], exercised)
       print(f"model.coef_ {model.intercept_} {model.coef_}")
       print(f"interpolated {interpolated}")
       print(f"exercise {exercised}")

    return np.mean(value), np.std(value)/np.sqrt(len(value))
    
def payoff_put(S,K): return np.maximum(K-S,0)

T = 3
dt =1 
r = 0.06
K = 1.10

val,std = least_square_Monte_Carlo_test(paths,r,dt,T,lambda S: payoff_put(S,K),K)
print(val,std)

[0.   0.   0.07 0.18 0.   0.2  0.09 0.  ]
model.coef_ -1.0699876552910912 [ 2.98341063 -1.81357618]
interpolated [0.03674056 0.04589834 0.11752682 0.15196921 0.15641792]
exercise [0.02 0.03 0.13 0.33 0.26]
model.coef_ 2.037512342379653 [-3.3354434   1.35645659]
interpolated [0.01348511 0.10874928 0.28606468 0.11700927 0.15276213]
exercise [0.01 0.17 0.34 0.18 0.22]
model.coef_ 0.11443433004505696 [0. 0.]
interpolated [0.11443433 0.11443433 0.11443433 0.11443433 0.11443433 0.11443433
 0.11443433 0.11443433]
exercise [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
0.11443433004505696 0.03922691623092562
